# 4 - Detecção de Overfitting - Conceitos

## 4.1 - O que é Overfitting
vamos falar sobre como detectar e evitar o overfitting em modelos de Machine Learning e AI. O overfitting ocorre quando um modelo se ajusta excessivamente aos dados de treinamento, mas não tem um bom desempenho em dados novos. Isso pode acontecer devido à complexidade do modelo, variáveis irrelevantes ou treinamento excessivo. Para evitar o overfitting, podemos aplicar técnicas como regularização, validação cruzada e simplificação do modelo. Vamos visualmente identificar modelos ótimos, com baixo erro de treinamento e teste, e modelos com underfitting e overfitting.

## 4.2 -KNN - Conceito
O algoritmo K-NN, ou K-Nearest Neighbors, é um método utilizado para classificação e regressão. Ele funciona com base na ideia de que instâncias de dados semelhantes estão próximas umas das outras em um espaço de características. Quando um novo ponto de dado é introduzido, o K-NN identifica os K pontos de dados mais próximos e determina sua classificação ou previsão com base na agregação dos valores ou classes desses vizinhos mais próximos. A escolha do valor de K e da métrica de distância é crucial para o desempenho do algoritmo. O K-NN é conhecido por sua simplicidade e eficácia em problemas de baixa a moderada dimensionalidade.

## 4.3 - Visualização Overfitting - Classificação
Nesta aula, aprendemos como detectar o overfitting em um modelo de classificação. Utilizamos o notebook para carregar os dados e ajustar o DataFrame. Em seguida, treinamos o modelo de regressão logística e o modelo de KNN (K vizinhos mais próximos). Utilizamos um loop para treinar o modelo com diferentes quantidades de vizinhos (K) e armazenamos os scores de treinamento e teste em listas separadas. Em seguida, criamos um gráfico para visualizar esses scores e identificar possíveis casos de overfitting. Observamos que, ao alterar o hiperparâmetro K, a diferença entre os scores de treinamento e teste pode diminuir ou aumentar. Isso nos ajuda a encontrar o conjunto de hiperparâmetros que oferece uma boa performance e evita o overfitting.

In [20]:
# Importando Bibliotecas

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

In [21]:
# Carregar Dataset
df2 = pd.read_csv('./fruit_quality.csv')
df2.head()

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,good
1,1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,good
2,2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,bad
3,3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,good
4,4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,good


In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   A_id         4000 non-null   int64  
 1   Size         4000 non-null   float64
 2   Weight       4000 non-null   float64
 3   Sweetness    4000 non-null   float64
 4   Crunchiness  4000 non-null   float64
 5   Juiciness    4000 non-null   float64
 6   Ripeness     4000 non-null   float64
 7   Acidity      4000 non-null   float64
 8   Quality      4000 non-null   object 
dtypes: float64(7), int64(1), object(1)
memory usage: 281.4+ KB


In [23]:
# Ajustar Data Frame

# Remover Coluna A_id, pois não tem poder preditivo
df2.drop(columns=['A_id'], axis=1, inplace=True)

# Tranformar a variável Quality em numérica (0 e 1)
# Convertendo pra int True se torna 1 e False 0
df2.Quality = (df2.Quality == 'good').astype(int)
df2

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,1
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,1
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,0
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,1
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,1
...,...,...,...,...,...,...,...,...
3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784,0
3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235,1
3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611,0
3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720,1


### 4.3.1 - Treinar modelo de KNN com mudança de K

In [24]:
# Treinar o modelo usando RFE

# Separar X e y
X = df2.drop('Quality', axis=1)
y = df2.Quality

In [25]:
# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [26]:
# Loop para treinar o modelo com diversas quantidades de vizinhos
scores_train = []
scores_test = []
for i in range(1, 20, 2):
    clf = KNeighborsClassifier(n_neighbors=i)
    clf.fit(X_train, y_train)
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    scores_train.append(f1_score(y_train, y_train_pred))
    scores_test.append(f1_score(y_test, y_test_pred))
    print(f'{i}: F1_Train: {f1_score(y_train, y_train_pred)} F1_test: {f1_score(y_test, y_test_pred)}')

1: F1_Train: 1.0 F1_test: 0.8740617180984154


3: F1_Train: 0.9452103216684341 F1_test: 0.8964941569282137
5: F1_Train: 0.9262266148958701 F1_test: 0.8985985160758451
7: F1_Train: 0.921935711762628 F1_test: 0.8929460580912864
9: F1_Train: 0.9186704384724187 F1_test: 0.8913043478260869
11: F1_Train: 0.9178130511463845 F1_test: 0.8887029288702929
13: F1_Train: 0.9115983026874116 F1_test: 0.8881469115191987
15: F1_Train: 0.9133969600565571 F1_test: 0.890728476821192
17: F1_Train: 0.9099258212645708 F1_test: 0.8883333333333333
19: F1_Train: 0.9115885875308207 F1_test: 0.885


In [27]:
# Criar um Data Frame com os Scores
df_results = pd.DataFrame({'k': range(1, 20, 2), 'train': scores_train, 'test': scores_test})
df_results

,k,train,test
0,1,1.000000,0.874062
1,3,0.945210,0.896494
2,5,0.926227,0.898599
3,7,0.921936,0.892946
4,9,0.918670,0.891304
5,11,0.917813,0.888703
6,13,0.911598,0.888147
7,15,0.913397,0.890728
8,17,0.909926,0.888333
9,19,0.911589,0.885000


In [28]:
# Plotar no gráfico os Scores
xaxis = x = [i for i in range(1, 20, 2)]
fig = px.line(df_results, x='k', y=['train', 'test'], title='KNN Performance - Mudando K')
fig.update_xaxes(tickvals=xaxis, ticktext=[str(i) for i in x])
fig.show()